In [3]:
import os
import pandas as pd

In [4]:
lab = pd.read_csv("C:/Users/rishi/Downloads/DCSASS Dataset/Labels/Shoplifting.csv")

In [5]:
lab.head(11)

,Shoplifting001_x264_0,Shoplifting,0
0,Shoplifting001_x264_1,Shoplifting,0
1,Shoplifting001_x264_2,Shoplifting,0
2,Shoplifting001_x264_3,Shoplifting,0
3,Shoplifting001_x264_4,Shoplifting,0
4,Shoplifting001_x264_5,Shoplifting,0
5,Shoplifting001_x264_6,Shoplifting,0
6,Shoplifting001_x264_7,Shoplifting,0
7,Shoplifting001_x264_8,Shoplifting,0
8,Shoplifting001_x264_9,Shoplifting,1
9,Shoplifting001_x264_10,Shoplifting,1


In [6]:
# Drop the 'Shoplifting' column
lab = lab.drop(columns=['Shoplifting'])

# Rename the '0' column to 'Shoplifting'
lab = lab.rename(columns={'0': 'Shoplifting'})

# Print the updated DataFrame
print(lab.head(11))


     Shoplifting001_x264_0  Shoplifting
0    Shoplifting001_x264_1            0
1    Shoplifting001_x264_2            0
2    Shoplifting001_x264_3            0
3    Shoplifting001_x264_4            0
4    Shoplifting001_x264_5            0
5    Shoplifting001_x264_6            0
6    Shoplifting001_x264_7            0
7    Shoplifting001_x264_8            0
8    Shoplifting001_x264_9            1
9   Shoplifting001_x264_10            1
10  Shoplifting001_x264_11            1


In [7]:
shoplifting_dir = 'C:/Users/rishi/Downloads/DCSASS Dataset/Labels/Shoplifting/'

# Initialize empty lists to store file paths and video names
file_paths = []
video_names = []


In [8]:
for root, dirs, files in os.walk(shoplifting_dir):
    for file in files:
        if file.endswith('.mp4'):  # Assuming videos are in MP4 format
            # Append video filename to the list of file paths
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
            # Extract video name from the full path
            video_name = os.path.splitext(file)[0]
            video_names.append(video_name)


In [9]:
df_shoplifting_videos = pd.DataFrame({'file_path': file_paths, 'video_name': video_names})

# Print the DataFrame
print(df_shoplifting_videos.head())

                                           file_path              video_name
0  /kaggle/input/dcsass-dataset/DCSASS Dataset/Sh...  Shoplifting045_x264_11
1  /kaggle/input/dcsass-dataset/DCSASS Dataset/Sh...   Shoplifting045_x264_5
2  /kaggle/input/dcsass-dataset/DCSASS Dataset/Sh...  Shoplifting045_x264_29
3  /kaggle/input/dcsass-dataset/DCSASS Dataset/Sh...  Shoplifting045_x264_27
4  /kaggle/input/dcsass-dataset/DCSASS Dataset/Sh...  Shoplifting045_x264_13


In [10]:
# Merge the two DataFrames based on the video names
combined_df = pd.merge(df_shoplifting_videos, lab, left_on='video_name', right_on='Shoplifting001_x264_0', how='inner')

# Drop the extra columns
combined_df.drop(columns=['Shoplifting001_x264_0'], inplace=True)

# Print the combined DataFrame
print(combined_df.head())


                                           file_path              video_name  \
0  /kaggle/input/dcsass-dataset/DCSASS Dataset/Sh...  Shoplifting045_x264_11   
1  /kaggle/input/dcsass-dataset/DCSASS Dataset/Sh...   Shoplifting045_x264_5   
2  /kaggle/input/dcsass-dataset/DCSASS Dataset/Sh...  Shoplifting045_x264_29   
3  /kaggle/input/dcsass-dataset/DCSASS Dataset/Sh...  Shoplifting045_x264_27   
4  /kaggle/input/dcsass-dataset/DCSASS Dataset/Sh...  Shoplifting045_x264_13   

   Shoplifting  
0            0  
1            0  
2            1  
3            1  
4            0  


In [11]:
combined_df.shape

(895, 3)

In [12]:
import cv2

In [13]:
import cv2
import numpy as np
import pandas as pd

# Define functions for preprocessing frames and processing videos
def preprocess_frame(frame, target_size):
    frame_resized = cv2.resize(frame, target_size)
    frame_normalized = frame_resized / 255.0
    return frame_normalized

def process_video(video_path, target_size=(59, 59), max_frames=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0

    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        frame_processed = preprocess_frame(frame, target_size)
        frames.append(frame_processed)

        frame_count += 1

    cap.release()

    while len(frames) < max_frames:
        frames.append(np.zeros_like(frames[0]))

    video_data = np.array(frames)

    return video_data



In [14]:
processed_data = []

# Apply preprocessing to each video in the DataFrame
for index, row in combined_df.iterrows():
    video_path = row['file_path']
    label = row['Shoplifting']
    processed_video = process_video(video_path)
    processed_data.append((processed_video, label))

# Convert the list of processed videos to a DataFrame
processed_df = pd.DataFrame(processed_data, columns=['video_data', 'label'])


In [15]:
print(processed_df.head())

                                          video_data  label
0  [[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...      0
1  [[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...      0
2  [[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...      1
3  [[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...      1
4  [[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...      0


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


2024-04-24 16:49:40.939159: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 16:49:40.939306: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 16:49:41.112919: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [17]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(processed_df['video_data'].values,
                                                    processed_df['label'].values,
                                                    test_size=0.2,
                                                    random_state=42)


In [18]:
from tensorflow.keras.layers import TimeDistributed, LSTM

# Define the CNN architecture
model = Sequential([
    TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(None, 59, 59, 3)),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Conv2D(64, (3, 3), activation='relu')),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Flatten()),
    LSTM(64),
    Dense(1, activation='sigmoid')
])



/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [19]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [21]:
# Stack frames along a new axis to create tensors for each sample
X_train = np.stack(X_train, axis=0)
X_test = np.stack(X_test, axis=0)

# Ensure that labels are in the correct format
y_train = np.array(y_train)
y_test = np.array(y_test)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)


Epoch 1/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 296s 13s/step - accuracy: 0.8005 - loss: 0.6099 - val_accuracy: 0.8750 - val_loss: 0.3740
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 289s 12s/step - accuracy: 0.8080 - loss: 0.4872 - val_accuracy: 0.8750 - val_loss: 0.3678
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 276s 13s/step - accuracy: 0.8108 - loss: 0.4749 - val_accuracy: 0.8750 - val_loss: 0.3891
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 298s 12s/step - accuracy: 0.8222 - loss: 0.4479 - val_accuracy: 0.8750 - val_loss: 0.3855
Epoch 5/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 277s 13s/step - accuracy: 0.8293 - loss: 0.4105 - val_accuracy: 0.8194 - val_loss: 0.3784
Epoch 6/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 246s 12s/step - accuracy: 0.8265 - loss: 0.3768 - val_accuracy: 0.8750 - val_loss: 0.3373
Epoch 7/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 282s 13s/step - accuracy: 0.8171 - loss: 0.3940 - val_accuracy: 0.8889 - val_loss: 0.3205
Epoch 8/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 302s 12s/step - accuracy: 0.8105 - loss: 0.4004 - val_accuracy: 0.

In [25]:
from sklearn.metrics import classification_report

# Assuming `model` has already been trained and X_test, y_test are defined
predictions_proba = model.predict(X_test)

# Convert probabilities to class labels using a threshold (e.g., 0.5)
threshold = 0.5
predictions = (predictions_proba > threshold).astype(int)

# Print classification report
print(classification_report(y_test, predictions))


6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 543ms/step
              precision    recall  f1-score   support

           0       0.89      0.94      0.91       154
           1       0.44      0.32      0.37        25

    accuracy                           0.85       179
   macro avg       0.67      0.63      0.64       179
weighted avg       0.83      0.85      0.84       179

